In [ ]:
import numpy as np
import scipy as scp
import math as ma
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sys
import os.path
import imp
import json
from scipy.optimize import curve_fit

import sklearn.cluster as skcltr
import sklearn.linear_model as skl_linear

import torch
import torch.nn as nn
import torch.nn.functional as nF
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# for data loading
import data_read as dr 

# for cross-validation data preparation 
import data_prep_crossvalid as dxv

# handmade linear regressors
import simpleLinearReg as slr

# import functions from other training methods
import GaussianLH_Panelty_RidgeLasso_MAP as glh_map

imp.reload(dr)
imp.reload(dxv)
imp.reload(slr)
imp.reload(glh_map)

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [ ]:
type(training_data)
# training_data.__dict__
type(training_data.data)
print(training_data.data.shape)
# training_data[0][0].shape
training_data_loader = DataLoader(training_data, batch_size=2, shuffle=False)
training_data_loader.__dict__
for i, item in enumerate(training_data_loader):
    print(type(item), ' : ', item )
    print(type(item[0]))
    print(item[0].shape)
    if i>10: break

In [ ]:
print(type(training_data_loader.dataset))
print(type(training_data))


In [ ]:
raw_data_tnsr = torch.tensor([[item for i in range(4)] for item in range(10)] )
print(raw_data_tnsr)
raw_dataset = torch.utils.data.TensorDataset(raw_data_tnsr)
print(raw_dataset)

In [ ]:
raw_data_loader = DataLoader(raw_dataset, batch_size=2, shuffle=1)
# raw_data_loader.__dict__

In [ ]:
for item in raw_data_loader:
    print(type(item), ' : ', item )
    print(type(item[0]))

In [75]:
raw_dataset.__dict__

{'tensors': (tensor([[0, 0, 0, 0],
          [1, 1, 1, 1],
          [2, 2, 2, 2],
          [3, 3, 3, 3],
          [4, 4, 4, 4],
          [5, 5, 5, 5],
          [6, 6, 6, 6],
          [7, 7, 7, 7],
          [8, 8, 8, 8],
          [9, 9, 9, 9]]),)}

In [76]:
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [78]:
for i, (images, labels) in enumerate(train_loader):
    if i>1: break

In [85]:
print(type(images))
print(images.shape)
aa = images.reshape(-1, 28*28)
print(aa.shape)

<class 'torch.Tensor'>
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])


In [86]:
labels

tensor([7, 6, 0, 2, 4, 6, 5, 6, 9, 1, 2, 5, 6, 9, 2, 8, 0, 1, 2, 0, 0, 7, 0, 9,
        6, 6, 2, 3, 8, 2, 1, 4, 0, 4, 7, 6, 7, 3, 7, 7, 0, 6, 9, 9, 9, 6, 4, 3,
        5, 9, 9, 8, 4, 4, 2, 7, 3, 8, 6, 2, 3, 2, 2, 5, 8, 9, 8, 9, 7, 8, 0, 2,
        7, 8, 8, 9, 6, 2, 5, 9, 1, 8, 5, 5, 6, 3, 6, 0, 9, 3, 8, 7, 3, 8, 9, 1,
        9, 0, 1, 0])

In [87]:
l1 = nn.Linear(4,1)
l2 = nn.Linear(4,2)

f_tnrs = torch.randn(3,4)

In [88]:
f_tnrs

tensor([[ 0.5721, -0.5435, -0.9314,  0.0944],
        [ 0.8518, -0.4941,  1.0365, -1.0864],
        [-1.3017,  0.4555,  0.3946, -1.3834]])

In [89]:
l1(f_tnrs)

tensor([[-0.2845],
        [ 0.9348],
        [ 1.0981]], grad_fn=<AddmmBackward0>)

In [94]:
print(l1.weight)
print(type(l1.weight))
print(l1.weight.__dict__)

Parameter containing:
tensor([[-0.3621, -0.4835,  0.4357, -0.4124]], requires_grad=True)
<class 'torch.nn.parameter.Parameter'>
{}


In [111]:
print((l1.weight[0].detach()*f_tnrs[2,:]).sum() + l1.bias[0].detach().item())
print(l1.bias[0].detach().item())

tensor(1.0981)
0.10465312004089355


In [113]:
y1=l1(f_tnrs)

In [112]:
lsfn = nn.MSELoss()
t_tnsr = torch.tensor([-0.2,0.9,1.0])

In [131]:
err = lsfn(y1, t_tnsr)
print(err)

tensor(0.6764, grad_fn=<MseLossBackward0>)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [118]:
print(y1)
print(t_tnsr)

tensor([[-0.2845],
        [ 0.9348],
        [ 1.0981]], grad_fn=<AddmmBackward0>)
tensor([-0.2000,  0.9000,  1.0000])


In [132]:
((y1-t_tnsr)**2).mean()

tensor(0.6764, grad_fn=<MeanBackward0>)

In [127]:
((y1[0,0]-t_tnsr[0])**2 + (y1[1,0]-t_tnsr[1])**2 + (y1[2,0]-t_tnsr[2])**2)/3

tensor(0.0060, grad_fn=<DivBackward0>)

In [129]:
err.item()

0.6763722896575928

In [146]:
x = torch.tensor([[i for j in range(3)] for i in range(6)], dtype=torch.float32)
y = torch.tensor([[i] for i in range(6)], dtype=torch.float32)

In [147]:
class myDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.f = x
        self.t = y
    def __len__(self):
        return len(self.f)
    def __getitem__(self, idx):
        sample = (self.f[idx], self.t[idx])
        return sample

In [148]:
myDset = myDataset(x, y)
print(myDset[1:3])

(tensor([[1., 1., 1.],
        [2., 2., 2.]]), tensor([[1.],
        [2.]]))


In [149]:
myloader = DataLoader(myDset, batch_size=2, shuffle=1)
for i, d in enumerate(myloader):
    print(d)

[tensor([[0., 0., 0.],
        [4., 4., 4.]]), tensor([[0.],
        [4.]])]
[tensor([[1., 1., 1.],
        [3., 3., 3.]]), tensor([[1.],
        [3.]])]
[tensor([[5., 5., 5.],
        [2., 2., 2.]]), tensor([[5.],
        [2.]])]


In [152]:
l3 = nn.Linear(3,1)
l3((myDset[0:3][0]))

tensor([[ 0.2637],
        [-0.4225],
        [-1.1086]], grad_fn=<AddmmBackward0>)